In [2]:
import pandas as pd
import numpy as np
import openpyxl  
import xlsxwriter

In [3]:
# Source file names
fee_files = r"C:\Users\iwalk\Downloads\Personal\Comp Exercise\Advisor Billing.xlsx"
rates_files = r"C:\Users\iwalk\Downloads\Personal\Comp Exercise\Advisor Metrics.xlsx"

#File Sheets
mgmt_df = pd.read_excel(fee_files, sheet_name="Management Bill")
sma_df = pd.read_excel(fee_files, sheet_name="SMA Fees")
charges_df = pd.read_excel(fee_files, sheet_name="Charges")

rates_df = pd.read_excel(rates_files, sheet_name="Rates")
splits_df = pd.read_excel(rates_files, sheet_name="Splits")

#Change format of account column
pd.set_option('display.float_format', '{:,.2f}'.format)


In [4]:
 #Remove commas and convert to float
mgmt_df['Fee'] = mgmt_df['Fee'].replace('[\$,]', '', regex=True).astype(float)
mgmt_df['Total Account Value'] = mgmt_df['Total Account Value'].replace('[\$,]', '', regex=True).astype(float)

In [5]:
# Unpivot Management df

mgmt_unpivoted = mgmt_df.melt(
    id_vars=['Acct#', 'Fee', 'Total Account Value', 'Split'],
    value_vars=['Service ', 'Sign', 'Intr'],
    var_name='Role',
    value_name='Advisor'
)



In [6]:
# Normalize Role values
mgmt_unpivoted['Role'] = mgmt_unpivoted['Role'].replace({'Service': 'Service', 'Sign': 'Sign', 'Intr': 'Intr'})

In [7]:
# get rid of trailing zeros
mgmt_unpivoted['Role'] = mgmt_unpivoted['Role'].str.strip()


In [8]:
#check for accuracy
mgmt_unpivoted['Role'].unique()

array(['Service', 'Sign', 'Intr'], dtype=object)

In [9]:
#unpivot SMA Fee data
sma_unpivoted = sma_df.melt(
    id_vars = ['Acct#','Fee','Split'],
    value_vars = ['Service ', 'Sign', 'Intr'],
    var_name = 'Role',
    value_name = 'Advisor'
)

In [10]:
sma_unpivoted['Role'] = sma_unpivoted['Role'].str.strip()
sma_unpivoted['Advisor'] = sma_unpivoted['Advisor'].str.strip()


In [11]:
sma_unpivoted['Role'].unique()

array(['Service', 'Sign', 'Intr'], dtype=object)

In [12]:
charges_unpivoted = charges_df.melt(
    id_vars = ['Acct#','Fee','Split'],
    value_vars = ['Service ', 'Sign', 'Intr'],
    var_name = 'Role',
    value_name = 'Advisor'
)

In [13]:
charges_unpivoted = charges_df.melt(
    id_vars = ['Acct#','Fee','Split'],
    value_vars = ['Service ', 'Sign', 'Intr'],
    var_name = 'Role',
    value_name = 'Advisor'
)

In [14]:
charges_unpivoted['Role'] = charges_unpivoted['Role'].str.strip()

In [15]:
charges_unpivoted['Role'].unique()

array(['Service', 'Sign', 'Intr'], dtype=object)

In [16]:
rates_df.columns = rates_df.columns.str.strip()
rates_df['Advisor'] = rates_df['Advisor'].str.strip()

In [17]:
rates_df

,Advisor,Advisor ID,Servicing,Signing,Intro
0,Jane Lane,1,0.20,0.12,0.03
1,Luke Davil,2,0.15,0.08,0.02
2,Chris Prirtell,3,0.18,0.06,0.05
3,Patik Sadiq,4,0.13,0.03,0.09
4,James Tama,5,0.22,0.11,0.05
5,Jennifer Kennigston,6,0.35,0.17,0.10
6,Fiona Ryder,7,0.10,0.05,0.03
7,Jacky Cambell,8,0.08,0.01,0.05
8,Brackton Backer,9,0.13,0.06,0.16


In [18]:
# unpivot rates to match Fee Df formats

rates_melted = pd.melt(
    rates_df,
    id_vars='Advisor',
    value_vars=['Servicing', 'Signing', 'Intro'],
    var_name='Role',
    value_name='Rate'
)

# Normalize Role names to match 'Service', 'Sign', 'Intr'
rates_melted['Role'] = rates_melted['Role'].replace({
    'Servicing': 'Service',
    'Signing': 'Sign',
    'Intro': 'Intr'
})

In [19]:
#Merge Dfs
mgmt_merged = mgmt_unpivoted.merge(rates_melted, on=['Advisor', 'Role'], how='left')

sma_merged = sma_unpivoted.merge(rates_melted, on=['Advisor', 'Role'], how='left')

charges_merged = charges_unpivoted.merge(rates_melted, on=['Advisor', 'Role'], how='left')
#print(charges_merged)

In [20]:
# Add Base commision column to each Df (Fee * Rate)
mgmt_merged['Base Commission'] = mgmt_merged['Fee'] * mgmt_merged['Rate']
sma_merged['Base Commission'] = sma_merged['Fee'] * sma_merged['Rate']
charges_merged['Base Commission'] = charges_merged['Fee'] * charges_merged['Rate']

#check for acc
1132927.8931 == mgmt_merged['Base Commission'].sum()

True

In [21]:
mgmt_merged = mgmt_merged.merge(
    splits_df,
    left_on='Split',
    right_on='Split Code',
    how='left'
)

In [22]:
sma_merged = sma_merged.merge(
    splits_df,
    left_on='Split',
    right_on='Split Code',
    how='left')

In [23]:
charges_merged = charges_merged.merge(
    splits_df,
    left_on='Split',
    right_on='Split Code',
    how='left'
)

In [24]:
# If split code is not null, change split percentage to 50% for cambell and backer
for df in [mgmt_merged, sma_merged, charges_merged]:
    df['Split %'] = df['Split Code'].apply(
        lambda x: 1 if pd.isna(x) else 0.5 if x in ['Cambell1', 'Backer1'] else None
    )

In [25]:
# Multiplying base commissions with split percentage for advisors with split (to add back later)
for df in [mgmt_merged, sma_merged, charges_merged]:
    df['Split Amount'] = df['Base Commission'] * df['Split %']

In [26]:
mgmt_merged.head()

,Acct#,Fee,Total Account Value,Split,Role,Advisor,Rate,Base Commission,Advisor to Split,Split Code,Split %,Split Amount
0,4580427,"4,647.75","2,446,184.21",NONE,Service,Jennifer Kennigston,0.35,"1,626.71",NaN,NaN,1.00,"1,626.71"
1,6761210,303.30,"514,052.74",NONE,Service,Jacky Cambell,0.08,24.26,NaN,NaN,1.00,24.26
2,8634780,919.61,"2,561,431.71",NONE,Service,Jacky Cambell,0.08,73.57,NaN,NaN,1.00,73.57
3,4163878,"1,616.05","778,295.29",NONE,Service,Chris Prirtell,0.18,290.89,NaN,NaN,1.00,290.89
4,9144236,"1,678.75","1,528,946.53",NONE,Service,Chris Prirtell,0.18,302.18,NaN,NaN,1.00,302.18


In [27]:
# If advisor has a split and the split is with themself, we take half of the fee and add it to their original commission)

for df in [mgmt_merged, sma_merged, charges_merged]:
    df['Split Amount'] = df['Base Commission'] * df['Split %']
    df['Final Commission'] = df.apply(
        lambda row: row['Split Amount'] + (row['Fee'] * row['Split %'])
        if (row['Split Code'] == 'Cambell1' and row['Advisor'] == 'Jacky Cambell')
        or (row['Split Code'] == 'Backer1' and row['Advisor'] == 'Brackton Backer')
        else row['Split Amount'],
        axis=1
    )

In [28]:
# Cleaning - Listing relevant columns
final_cols = [
    'Advisor',
    'Acct#',
    'Total Account Value',
    'Role',
    'Fee',
    'Split Code',
    'Rate',
    'Split %',
    'Base Commission',
    'Split Amount',
    'Final Commission'
]

# Filter and reorder mgmt_merged accordingly
mgmt_clean = mgmt_merged[final_cols]

In [29]:
mgmt_final = mgmt_clean  

In [30]:
sma_final_cols = [
    'Advisor',
    'Acct#',
    'Role',
    'Fee',
    'Split Code',
    'Rate',
    'Split %',
    'Base Commission',
    'Split Amount',
    'Final Commission'
]

sma_final = sma_merged[sma_final_cols]

In [31]:
charges_final_cols = [
    'Advisor',
    'Acct#',
    'Role',
    'Fee',
    'Split Code',
    'Rate',
    'Split %',
    'Base Charge',
    'Split Amount',
    'Final Charge'
]
# Renaming base and final commission to base and final charge
charges_merged = charges_merged.rename(columns={
    'Base Commission': 'Base Charge',
    'Final Commission': 'Final Charge'
})

# Reordering & keep relevant columns
charges_final = charges_merged[charges_final_cols]


In [32]:
charges_final.head()

,Advisor,Acct#,Role,Fee,Split Code,Rate,Split %,Base Charge,Split Amount,Final Charge
0,Chris Prirtell,4830049,Service,823.60,Cambell1,0.18,0.50,148.25,74.12,74.12
1,Luke Davil,9748094,Service,872.97,NaN,0.15,1.00,130.95,130.95,130.95
2,Jacky Cambell,3163812,Service,4.39,NaN,0.08,1.00,0.35,0.35,0.35
3,Chris Prirtell,2822293,Service,"1,159.04",NaN,0.18,1.00,208.63,208.63,208.63
4,Jacky Cambell,1660261,Service,32.12,NaN,0.08,1.00,2.57,2.57,2.57


In [33]:
# Get list of all advisors to create spreadsheet for
unique_advisors = rates_df['Advisor'].dropna().unique()

for advisor_name in unique_advisors:
    # Filter dataframes
    mgmt_filtered = mgmt_final[mgmt_final['Advisor'] == advisor_name]
    sma_filtered = sma_final[sma_final['Advisor'] == advisor_name]
    charges_filtered = charges_final[charges_final['Advisor'] == advisor_name]

    # Calculate totals for summary and details sheet
    mgmt_total = mgmt_filtered['Final Commission'].sum()
    sma_total = sma_filtered['Final Commission'].sum()
    charges_total = charges_filtered['Final Charge'].sum()
    payout_total = mgmt_total + sma_total - charges_total

    # Build output filename
    output_filename = f"Payout_{advisor_name.replace(' ', '_')}_Final.xlsx"

    with pd.ExcelWriter(output_filename, engine='xlsxwriter') as writer:
        # Write full DataFrames (all advisors) to their respective sheets
        mgmt_final.to_excel(writer, sheet_name='Management Bill', index=False)
        sma_final.to_excel(writer, sheet_name='SMA Fees', index=False)
        charges_final.to_excel(writer, sheet_name='Charges', index=False)

        workbook = writer.book

        # ------------- Commission_Detail SHEET -------------
        ws = workbook.add_worksheet('Commission_Detail')
        writer.sheets['Commission_Detail'] = ws

        # Formats
        yellow = workbook.add_format({'bg_color': '#FFFF00', 'bold': True, 'align': 'center'})
        money = workbook.add_format({'num_format': '$#,##0.00'})
        bold = workbook.add_format({'bold': True})

        # --- 1. Management Bill Section (starts at A4) ---
        ws.write('A3', 'Management Bill', yellow)
        mgmt_filtered.to_excel(writer, sheet_name='Commission_Detail', startrow=4, startcol=0, index=False, header=True)
        ws.write('K2', mgmt_total, money)  # Total above last column

        # --- 2. SMA Fees Section (starts at N4) ---
        ws.write('N3', 'SMA Fees', yellow)
        sma_filtered.to_excel(writer, sheet_name='Commission_Detail', startrow=4, startcol=13, index=False, header=True)
        ws.write('W2', sma_total, money)  # Total above last column

        # --- 3. Charges Section (starts at AA4) ---
        ws.write('AA3', 'Charges', yellow)
        charges_filtered.to_excel(writer, sheet_name='Commission_Detail', startrow=4, startcol=26, index=False, header=True)
        ws.write('AJ2', charges_total, money)  # Total above last column

        # ------------- SUMMARY SHEET -------------
        ws_sum = workbook.add_worksheet('Summary')
        writer.sheets['Summary'] = ws_sum

        # Layout per your image
        ws_sum.write('C2', advisor_name, yellow)
        ws_sum.write('C3', 'Template', bold)
        ws_sum.write('C4', 'COMMISSION SUMMARY', bold)
        ws_sum.write('C5', 'DATE')

        ws_sum.write('E8', 'Notes', bold)
        ws_sum.write('C10', 'MANAGEMENT FEES', bold)
        ws_sum.write('C12', 'SMA FEES', bold)
        ws_sum.write('C14', 'CHARGES', bold)
        ws_sum.write('C18', 'PAYOUT', bold)

        # Link to Commission_Detail totals
        ws_sum.write_formula('D10', "=Commission_Detail!K2")
        ws_sum.write_formula('D12', "=Commission_Detail!W2")
        ws_sum.write_formula('D14', "=-Commission_Detail!AJ2")

        # Payout = sum of above (or D10+D12+D14)
        ws_sum.write_formula('D18', "=SUM(D10:D14)", money)

        # (Optional) Formatting for the payout cell
        ws_sum.set_row(17, None, bold)  # Row 18 = index 17

    print(f"Workbook created for {advisor_name}: {output_filename}")

Workbook created for Jane Lane: Payout_Jane_Lane_Final.xlsx
Workbook created for Luke Davil: Payout_Luke_Davil_Final.xlsx
Workbook created for Chris Prirtell: Payout_Chris_Prirtell_Final.xlsx
Workbook created for Patik Sadiq: Payout_Patik_Sadiq_Final.xlsx
Workbook created for James Tama: Payout_James_Tama_Final.xlsx
Workbook created for Jennifer Kennigston: Payout_Jennifer_Kennigston_Final.xlsx
Workbook created for Fiona Ryder: Payout_Fiona_Ryder_Final.xlsx
Workbook created for Jacky Cambell: Payout_Jacky_Cambell_Final.xlsx
Workbook created for Brackton Backer: Payout_Brackton_Backer_Final.xlsx
